# CAT BOOST

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [2]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [3]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [4]:
values["building_id"].count() == values["building_id"].drop_duplicates().count()

True

In [5]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")

In [6]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [7]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)

In [8]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,0,0,2


In [9]:
X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [10]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [11]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from catboost import CatBoostClassifier

In [24]:
model = CatBoostClassifier( iterations = 7500,
                            eval_metric='Accuracy',
                            loss_function='MultiClass',
                            use_best_model=True,
                            random_seed=1,
                            max_depth = None,
#                             learning_rate = 0.0625,
                            boost_from_average = False,
                            verbose=True)
model.fit(X_train, y_train, 
          cat_features=['geo_level_2_id', 'geo_level_3_id'],
          eval_set=(X_train,y_train))

Learning rate set to 0.055646
0:	learn: 0.6930305	test: 0.6992565	best: 0.6992565 (0)	total: 127ms	remaining: 15m 52s
1:	learn: 0.6993956	test: 0.7052379	best: 0.7052379 (1)	total: 258ms	remaining: 16m 7s
2:	learn: 0.6991462	test: 0.7047583	best: 0.7052379 (1)	total: 354ms	remaining: 14m 45s
3:	learn: 0.6996163	test: 0.7056168	best: 0.7056168 (3)	total: 438ms	remaining: 13m 41s
4:	learn: 0.6992421	test: 0.7050796	best: 0.7056168 (3)	total: 553ms	remaining: 13m 49s
5:	learn: 0.6992661	test: 0.7051036	best: 0.7056168 (3)	total: 634ms	remaining: 13m 12s
6:	learn: 0.7008682	test: 0.7067105	best: 0.7067105 (6)	total: 744ms	remaining: 13m 15s
7:	learn: 0.7003406	test: 0.7061205	best: 0.7067105 (6)	total: 851ms	remaining: 13m 16s
8:	learn: 0.7006092	test: 0.7062788	best: 0.7067105 (6)	total: 942ms	remaining: 13m 4s
9:	learn: 0.7016213	test: 0.7073916	best: 0.7073916 (9)	total: 1.06s	remaining: 13m 15s
10:	learn: 0.7021441	test: 0.7080056	best: 0.7080056 (10)	total: 1.16s	remaining: 13m 9s
11:

92:	learn: 0.7286598	test: 0.7468678	best: 0.7468678 (92)	total: 11.2s	remaining: 14m 51s
93:	learn: 0.7289092	test: 0.7472995	best: 0.7472995 (93)	total: 11.3s	remaining: 14m 48s
94:	learn: 0.7289620	test: 0.7473762	best: 0.7473762 (94)	total: 11.5s	remaining: 14m 53s
95:	learn: 0.7290483	test: 0.7474242	best: 0.7474242 (95)	total: 11.6s	remaining: 14m 53s
96:	learn: 0.7291203	test: 0.7478895	best: 0.7478895 (96)	total: 11.7s	remaining: 14m 53s
97:	learn: 0.7291922	test: 0.7479279	best: 0.7479279 (97)	total: 11.9s	remaining: 14m 59s
98:	learn: 0.7293266	test: 0.7481485	best: 0.7481485 (98)	total: 12s	remaining: 14m 56s
99:	learn: 0.7293505	test: 0.7481533	best: 0.7481533 (99)	total: 12.1s	remaining: 14m 56s
100:	learn: 0.7294896	test: 0.7483212	best: 0.7483212 (100)	total: 12.2s	remaining: 14m 54s
101:	learn: 0.7295712	test: 0.7483020	best: 0.7483212 (100)	total: 12.4s	remaining: 14m 56s
102:	learn: 0.7296335	test: 0.7484315	best: 0.7484315 (102)	total: 12.4s	remaining: 14m 53s
103:	l

184:	learn: 0.7340656	test: 0.7538421	best: 0.7538565 (182)	total: 23.6s	remaining: 15m 33s
185:	learn: 0.7340416	test: 0.7540436	best: 0.7540436 (185)	total: 23.7s	remaining: 15m 33s
186:	learn: 0.7341184	test: 0.7540388	best: 0.7540436 (185)	total: 23.8s	remaining: 15m 32s
187:	learn: 0.7341616	test: 0.7539956	best: 0.7540436 (185)	total: 24s	remaining: 15m 33s
188:	learn: 0.7342143	test: 0.7540579	best: 0.7540579 (188)	total: 24.1s	remaining: 15m 33s
189:	learn: 0.7342143	test: 0.7541587	best: 0.7541587 (189)	total: 24.3s	remaining: 15m 33s
190:	learn: 0.7342191	test: 0.7541491	best: 0.7541587 (189)	total: 24.4s	remaining: 15m 32s
191:	learn: 0.7342527	test: 0.7542066	best: 0.7542066 (191)	total: 24.5s	remaining: 15m 31s
192:	learn: 0.7343007	test: 0.7542354	best: 0.7542354 (192)	total: 24.6s	remaining: 15m 31s
193:	learn: 0.7342623	test: 0.7542690	best: 0.7542690 (193)	total: 24.8s	remaining: 15m 32s
194:	learn: 0.7342911	test: 0.7543361	best: 0.7543361 (194)	total: 24.9s	remaining

275:	learn: 0.7371163	test: 0.7572669	best: 0.7572957 (269)	total: 36.2s	remaining: 15m 47s
276:	learn: 0.7372362	test: 0.7573388	best: 0.7573388 (276)	total: 36.4s	remaining: 15m 47s
277:	learn: 0.7371930	test: 0.7573628	best: 0.7573628 (277)	total: 36.5s	remaining: 15m 47s
278:	learn: 0.7372506	test: 0.7574683	best: 0.7574683 (278)	total: 36.6s	remaining: 15m 47s
279:	learn: 0.7372937	test: 0.7575115	best: 0.7575115 (279)	total: 36.7s	remaining: 15m 47s
280:	learn: 0.7372458	test: 0.7575691	best: 0.7575691 (280)	total: 36.9s	remaining: 15m 47s
281:	learn: 0.7373609	test: 0.7575835	best: 0.7575835 (281)	total: 37s	remaining: 15m 46s
282:	learn: 0.7373801	test: 0.7576026	best: 0.7576026 (282)	total: 37.1s	remaining: 15m 46s
283:	learn: 0.7374328	test: 0.7576842	best: 0.7576842 (283)	total: 37.3s	remaining: 15m 46s
284:	learn: 0.7374616	test: 0.7577178	best: 0.7577178 (284)	total: 37.4s	remaining: 15m 46s
285:	learn: 0.7374424	test: 0.7577993	best: 0.7577993 (285)	total: 37.6s	remaining

366:	learn: 0.7393947	test: 0.7596316	best: 0.7596316 (366)	total: 49.7s	remaining: 16m 5s
367:	learn: 0.7395290	test: 0.7596220	best: 0.7596316 (366)	total: 49.8s	remaining: 16m 4s
368:	learn: 0.7394570	test: 0.7596124	best: 0.7596316 (366)	total: 49.9s	remaining: 16m 4s
369:	learn: 0.7394714	test: 0.7595980	best: 0.7596316 (366)	total: 50s	remaining: 16m 3s
370:	learn: 0.7395146	test: 0.7596700	best: 0.7596700 (370)	total: 50.2s	remaining: 16m 3s
371:	learn: 0.7395578	test: 0.7596892	best: 0.7596892 (371)	total: 50.3s	remaining: 16m 3s
372:	learn: 0.7395482	test: 0.7597036	best: 0.7597036 (372)	total: 50.4s	remaining: 16m 2s
373:	learn: 0.7395721	test: 0.7597276	best: 0.7597276 (373)	total: 50.5s	remaining: 16m 2s
374:	learn: 0.7395482	test: 0.7597276	best: 0.7597276 (373)	total: 50.6s	remaining: 16m 2s
375:	learn: 0.7395913	test: 0.7597707	best: 0.7597707 (375)	total: 50.8s	remaining: 16m 3s
376:	learn: 0.7396441	test: 0.7598187	best: 0.7598187 (376)	total: 51s	remaining: 16m 2s
377

...

7122:	learn: 0.7813172	test: 0.7883154	best: 0.7883442 (7116)	total: 21m 6s	remaining: 1m 7s
7123:	learn: 0.7813363	test: 0.7883154	best: 0.7883442 (7116)	total: 21m 6s	remaining: 1m 6s
7124:	learn: 0.7813315	test: 0.7883154	best: 0.7883442 (7116)	total: 21m 6s	remaining: 1m 6s
7125:	learn: 0.7813267	test: 0.7883106	best: 0.7883442 (7116)	total: 21m 6s	remaining: 1m 6s
7126:	learn: 0.7813603	test: 0.7883010	best: 0.7883442 (7116)	total: 21m 6s	remaining: 1m 6s
7127:	learn: 0.7813603	test: 0.7882914	best: 0.7883442 (7116)	total: 21m 7s	remaining: 1m 6s
7128:	learn: 0.7813603	test: 0.7882866	best: 0.7883442 (7116)	total: 21m 7s	remaining: 1m 5s
7129:	learn: 0.7813411	test: 0.7882675	best: 0.7883442 (7116)	total: 21m 7s	remaining: 1m 5s
7130:	learn: 0.7813651	test: 0.7882531	best: 0.7883442 (7116)	total: 21m 7s	remaining: 1m 5s
7131:	learn: 0.7813459	test: 0.7882579	best: 0.7883442 (7116)	total: 21m 7s	remaining: 1m 5s
7132:	learn: 0.7813603	test: 0.7882962	best: 0.7883442 (7116)	total: 2

7211:	learn: 0.7815906	test: 0.7885649	best: 0.7885696 (7210)	total: 21m 22s	remaining: 51.2s
7212:	learn: 0.7815666	test: 0.7885792	best: 0.7885792 (7212)	total: 21m 22s	remaining: 51s
7213:	learn: 0.7815762	test: 0.7885792	best: 0.7885792 (7212)	total: 21m 22s	remaining: 50.9s
7214:	learn: 0.7815810	test: 0.7885936	best: 0.7885936 (7214)	total: 21m 23s	remaining: 50.7s
7215:	learn: 0.7815906	test: 0.7885792	best: 0.7885936 (7214)	total: 21m 23s	remaining: 50.5s
7216:	learn: 0.7816289	test: 0.7886032	best: 0.7886032 (7216)	total: 21m 23s	remaining: 50.3s
7217:	learn: 0.7816961	test: 0.7886224	best: 0.7886224 (7217)	total: 21m 23s	remaining: 50.2s
7218:	learn: 0.7817249	test: 0.7886560	best: 0.7886560 (7218)	total: 21m 23s	remaining: 50s
7219:	learn: 0.7816865	test: 0.7886512	best: 0.7886560 (7218)	total: 21m 24s	remaining: 49.8s
7220:	learn: 0.7817009	test: 0.7886272	best: 0.7886560 (7218)	total: 21m 24s	remaining: 49.6s
7221:	learn: 0.7817057	test: 0.7886224	best: 0.7886560 (7218)	to

7299:	learn: 0.7821757	test: 0.7887855	best: 0.7888239 (7281)	total: 21m 38s	remaining: 35.6s
7300:	learn: 0.7821710	test: 0.7887999	best: 0.7888239 (7281)	total: 21m 38s	remaining: 35.4s
7301:	learn: 0.7821470	test: 0.7887903	best: 0.7888239 (7281)	total: 21m 39s	remaining: 35.2s
7302:	learn: 0.7821278	test: 0.7887999	best: 0.7888239 (7281)	total: 21m 39s	remaining: 35s
7303:	learn: 0.7821566	test: 0.7888335	best: 0.7888335 (7303)	total: 21m 39s	remaining: 34.9s
7304:	learn: 0.7821662	test: 0.7888239	best: 0.7888335 (7303)	total: 21m 39s	remaining: 34.7s
7305:	learn: 0.7821710	test: 0.7888191	best: 0.7888335 (7303)	total: 21m 39s	remaining: 34.5s
7306:	learn: 0.7821757	test: 0.7888095	best: 0.7888335 (7303)	total: 21m 39s	remaining: 34.3s
7307:	learn: 0.7821710	test: 0.7887999	best: 0.7888335 (7303)	total: 21m 40s	remaining: 34.2s
7308:	learn: 0.7821805	test: 0.7887951	best: 0.7888335 (7303)	total: 21m 40s	remaining: 34s
7309:	learn: 0.7821757	test: 0.7887951	best: 0.7888335 (7303)	to

7388:	learn: 0.7826842	test: 0.7890445	best: 0.7890445 (7388)	total: 21m 55s	remaining: 19.8s
7389:	learn: 0.7826890	test: 0.7890301	best: 0.7890445 (7388)	total: 21m 55s	remaining: 19.6s
7390:	learn: 0.7826986	test: 0.7890061	best: 0.7890445 (7388)	total: 21m 55s	remaining: 19.4s
7391:	learn: 0.7826890	test: 0.7890205	best: 0.7890445 (7388)	total: 21m 55s	remaining: 19.2s
7392:	learn: 0.7826890	test: 0.7890205	best: 0.7890445 (7388)	total: 21m 55s	remaining: 19s
7393:	learn: 0.7827130	test: 0.7890205	best: 0.7890445 (7388)	total: 21m 56s	remaining: 18.9s
7394:	learn: 0.7827226	test: 0.7890109	best: 0.7890445 (7388)	total: 21m 56s	remaining: 18.7s
7395:	learn: 0.7827130	test: 0.7890109	best: 0.7890445 (7388)	total: 21m 56s	remaining: 18.5s
7396:	learn: 0.7827082	test: 0.7890253	best: 0.7890445 (7388)	total: 21m 56s	remaining: 18.3s
7397:	learn: 0.7827082	test: 0.7890157	best: 0.7890445 (7388)	total: 21m 56s	remaining: 18.2s
7398:	learn: 0.7826986	test: 0.7890349	best: 0.7890445 (7388)	

7478:	learn: 0.7830104	test: 0.7892172	best: 0.7892172 (7478)	total: 22m 11s	remaining: 3.74s
7479:	learn: 0.7830343	test: 0.7892316	best: 0.7892316 (7479)	total: 22m 11s	remaining: 3.56s
7480:	learn: 0.7830343	test: 0.7892652	best: 0.7892652 (7480)	total: 22m 12s	remaining: 3.38s
7481:	learn: 0.7830391	test: 0.7892939	best: 0.7892939 (7481)	total: 22m 12s	remaining: 3.21s
7482:	learn: 0.7830583	test: 0.7892508	best: 0.7892939 (7481)	total: 22m 12s	remaining: 3.03s
7483:	learn: 0.7830631	test: 0.7892891	best: 0.7892939 (7481)	total: 22m 12s	remaining: 2.85s
7484:	learn: 0.7830583	test: 0.7892700	best: 0.7892939 (7481)	total: 22m 12s	remaining: 2.67s
7485:	learn: 0.7830631	test: 0.7892556	best: 0.7892939 (7481)	total: 22m 12s	remaining: 2.49s
7486:	learn: 0.7830775	test: 0.7892508	best: 0.7892939 (7481)	total: 22m 13s	remaining: 2.31s
7487:	learn: 0.7830535	test: 0.7892556	best: 0.7892939 (7481)	total: 22m 13s	remaining: 2.14s
7488:	learn: 0.7830295	test: 0.7892364	best: 0.7892939 (7481

In [25]:
model.learning_rate_

0.05564599856734276

In [26]:
model_pred = model.predict(X_test)
f1_score(y_test, model_pred, average='micro')

0.749294909921145

In [52]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")

In [53]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")

In [54]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)

In [55]:
preds = model.predict(test_values_subset)

In [56]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [57]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [58]:
my_submission.to_csv('../../csv/predictions/jf/cat-boost/jf-model-3-prueba-submission.csv')